# TODO GO to KRM and create a volume first with name "volume-sentinel"

In krm create first a volume otherwise the RUN is stuck waiting for the volume
persisten_volume_claim specs:
- volume_name : volume-sentinel # this name is to be equal to the "name parameter inside volumes during the run of the function"
- space : 15Gbi
- storage_class_name: #disk
- access_mode: readWriteOnce

If you have any doubts on the params check the README.md inside https://github.com/tn-aixpa/sentinel-tools 

In [1]:
import digitalhub as dh
# conda install -c conda-forge gdal
PROJECT_NAME = "deforestation"
proj = dh.get_or_create_project(PROJECT_NAME) # source="git://github.com/scc-digitalhu

Register to the open data space copenicus(if not already) and get your credentials.

https://identity.dataspace.copernicus.eu/auth/realms/CDSE/login-actions/registration?client_id=cdse-public&tab_id=FIiRPJeoiX4

Log the credentials as project secret keys as shown below

In [ ]:
# THIS NEED TO BE EXECUTED JUST ONCE
secret0 = proj.new_secret(name="CDSETOOL_ESA_USER", secret_value="esa_username")
secret1 = proj.new_secret(name="CDSETOOL_ESA_PASSWORD", secret_value="esa_password")

### Download data from Sentinel 2

In [4]:
string_dict_data = """{
 "satelliteParams":{
     "satelliteType": "Sentinel2"
 },
 "startDate": "2018-01-01",
 "endDate": "2019-12-31",
 "geometry": "POLYGON((10.98014831542969 45.455314263477874,11.030273437500002 45.44808893044964,10.99937438964844 45.42014226680115,10.953025817871096 45.435803739956725,10.98014831542969 45.455314263477874))",
 "area_sampling": "true",
 "cloudCover": "[0,20]",
 "artifact_name": "s2data_deforstation_2018_19" 
 }"""

list_args =  ["main.py",string_dict_data]
function_s2 = proj.new_function("download_images_s2",kind="container",image="ghcr.io/tn-aixpa/sentinel-tools:0.10.1",command="python")

In [5]:
run = function_s2.run(action="job",
        secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
        fs_group='8877',
        args=list_args,           
        volumes=[{
            "volume_type": "persistent_volume_claim",            
            "name": "volume-s2", # this name has to be equal to the name of the volume created in krm
            "mount_path": "/app/files",
            "spec": {
                "claim_name": "volume-s2"
    }}],)

In [6]:
run.refresh().status.state

'RUNNING'

In [3]:
artifact = proj.get_artifact('s2data_deforstation_2018_19')

### Perform Data Elaboration on RS-Tool

In [ ]:
function_rs = proj.new_function("elaborate",kind="container",image="ghcr.io/tn-aixpa/rsde-tools:0.1_beta2",command="python")

In [10]:
run_el = function_rs.run(action="job",
        #secrets=["CDSETOOL_ESA_USER","CDSETOOL_ESA_PASSWORD"],
        #fs_group='8877',
        args=['--version'])

In [11]:
run_el.refresh().status.state

'RUNNING'